In [1]:
import pandas as pd 
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
rating_data = pd.read_csv("../Data/ratings.csv")
movies_data = pd.read_csv("../Data/movies.csv")
tags_data = pd.read_csv("../Data/tags.csv")

In [3]:
movies_data.insert(2, "year",  movies_data["title"].str.extract(r"\((\d{4})\)"))
movies_data["title"] = movies_data["title"].str.replace(r"(\s*\(\d{4})\)", "", regex=True)

In [6]:
genres = ["Action", "Adventure", "Animation", "Children", "Comedy", 
          "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", 
          "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", 
          "War", "Western", "(no genres listed)"]
        
vectorizer = CountVectorizer()
vectorizer.fit(genres)
bow_input = movies_data[["movieId", "genres"]].copy()
bow_input["BoW_input"] = bow_input["genres"].apply( lambda row: vectorizer.transform([row]).toarray())
bow_input.drop(["genres"], axis=1, inplace=True)

bow_input

,movieId,BoW_input
0,1,"[[0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,..."
1,2,"[[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,..."
2,3,"[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,4,"[[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."
4,5,"[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...
86532,288967,"[[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."
86533,288971,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
86534,288975,"[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
86535,288977,"[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [7]:
tags_data["tag"] = tags_data["tag"].astype(str) 
tags = tags_data.groupby(["movieId"], as_index=False).aggregate({"tag" : " ".join})
tags["tag"] = tags["tag"].str.replace(" ", "|")
movies_data = movies_data.merge(tags, on="movieId", how="left")
movies_data["tag"] = movies_data["tag"].fillna("")
movies_data["tf_idf_input"] = movies_data["genres"] + " " + movies_data["tag"]

movies_data = movies_data.drop("tag", axis=1)

movies_data

,movieId,title,year,genres,tf_idf_input
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,Adventure|Animation|Children|Comedy|Fantasy an...
1,2,Jumanji,1995,Adventure|Children|Fantasy,Adventure|Children|Fantasy animals|based|on|a|...
2,3,Grumpier Old Men,1995,Comedy|Romance,Comedy|Romance sequel|moldy|old|old|age|old|me...
3,4,Waiting to Exhale,1995,Comedy|Drama|Romance,Comedy|Drama|Romance characters|chick|flick|gi...
4,5,Father of the Bride Part II,1995,Comedy,Comedy family|pregnancy|wedding|4th|wall|aging...
...,...,...,...,...,...
86532,288967,State of Siege: Temple Attack,2021,Action|Drama,Action|Drama
86533,288971,Ouija Japan,2021,Action|Horror,Action|Horror
86534,288975,The Men Who Made the Movies: Howard Hawks,1973,Documentary,Documentary
86535,288977,Skinford: Death Sentence,2023,Crime|Thriller,Crime|Thriller


In [8]:
genres = ["Action", "Adventure", "Animation", "Children", "Comedy", 
          "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", 
          "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", 
          "War", "Western", "(no genres listed)"]

df = movies_data

vectorizer = CountVectorizer()
vectorizer.fit(genres)
df["BoW_input"] = df["genres"].apply( lambda row: vectorizer.transform([row]).toarray())

df

KeyboardInterrupt: 

In [ ]:
rating_data

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119
...,...,...,...,...
33832157,330975,8340,2.0,1091583256
33832158,330975,8493,2.5,1091585709
33832159,330975,8622,4.0,1091581777
33832160,330975,8665,3.0,1091581765


In [ ]:
time_tags = tags_data.drop(["userId", "tag"],axis=1)
# time_tags.rename({"timestamp":"tag_timestamp"}, inplace=True)
time_rating = rating_data.drop(["userId", "rating"], axis=1)
# time_rating.rename({"timestamp":"rating_timestamp"}, inplace=True)
print(len(time_rating),len(time_tags))

time_data = pd.concat([time_tags, time_rating])



33832162 2328315


In [ ]:
time_data.groupby(["movieId"]).max()
time_data.groupby(["movieId"]).min()
print(time_data.groupby(["movieId"]).sum())
print(time_data.groupby(["movieId"]).count())

              timestamp
movieId                
1        92800244625366
2        35984924322279
3        15536719885246
4         2893165593814
5        16054159106026
...                 ...
288967       1689748357
288971       1689798322
288975       1689812351
288977       1689815902
288983       1689834886

[86537 rows x 1 columns]
         timestamp
movieId           
1            78253
2            30862
3            15856
4             3041
5            15869
...            ...
288967           1
288971           1
288975           1
288977           1
288983           1

[86537 rows x 1 columns]


In [ ]:

mask = time_data.groupby(["movieId"]).count() < 2
mask["timestamp"].value_counts()

timestamp
False    72157
True     14380
Name: count, dtype: int64

In [ ]:
#     ** since_release_to_mean - Days from movie release to mean timestamp
#     ** last_rating - How recently the movie was engaged with
#     ** engagement_span - Hur långt från första till senaste.
#     ** mean_rating_time - Average time when ratings happened
#     ** std_rating_time	- Spread/consistency of rating timestamps
#     ** total_engagement - Totalt antalet tags och ratings.

feature_list = [ "mean_time", "std_time", "last_engagement", "total_engagement", "release_year_unix", "since_release_to_mean", "engagement_span"]
time_data_features = time_data.groupby("movieId")["timestamp"].agg(["mean", "std", "min", "max", "count"]).reset_index()
time_data_features.columns = ["movieId", "mean_time", "std_time", "min_time", "last_engagement", "total_engagement"]
time_data_features = time_data_features.merge(
    movies_data[["movieId", "year"]],
    on="movieId",
    how="left"
)

time_data_features["release_date"] = pd.to_datetime(movies_data["year"], format="%Y")
time_data_features["release_year_unix"] = (time_data_features["release_date"] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
time_data_features["since_release_to_mean"] = time_data_features["mean_time"] - time_data_features["release_date_unix"]
time_data_features["engagement_span"] = time_data_features["last_engagement"] - time_data_features["min_time"]
time_data_features.drop(["year", "release_date", "min_time"], axis=1, inplace=True)

scaler = MinMaxScaler()
time_data_features[feature_list] = scaler.fit_transform(time_data_features[feature_list])
time_data_features = time_data_features[time_data_features['total_engagement'] > 0]
time_data_features.dropna(inplace=True)
time_data_features



KeyError: 'release_date_unix'

In [ ]:
engagement_cluster = KMeans(n_clusters=5)
engagement_cluster.fit(time_data_features[feature_list])

time_data_features["clusters"] =  engagement_cluster.labels_

cluster_means = time_data_features.groupby("clusters")[feature_list].mean()
sorted_clusters = cluster_means.sort_values(by="std_time").index

labels = ["Very Low", "Low", "Medium", "High", "Very High"]
class_labels = {cluster : labels[i] for i, cluster in enumerate(sorted_clusters)}

time_data_features["engagement_level"] = time_data_features["clusters"].map(class_labels)
time_data_features.drop("clusters", axis=1, inplace=True)

time_data_features.sort_values(by="std_time")


,movieId,mean_time,std_time,last_engagement,total_engagement,release_date_unix,since_release_to_mean,engagement_span,engagement_level
24299,121065,0.802608,0.000000,0.752480,0.000008,0.409382,0.574446,0.000000,Low
25478,123697,0.802524,0.000000,0.752374,0.000031,0.523474,0.457308,0.000000,Low
25475,123691,0.802524,0.000000,0.752374,0.000015,0.570460,0.409075,0.000000,Low
25453,123643,0.802524,0.000000,0.752374,0.000008,0.825509,0.147251,0.000000,Very Low
42087,162144,0.802524,0.000000,0.752375,0.000023,0.409382,0.574431,0.000000,Low
...,...,...,...,...,...,...,...,...,...
6555,6677,0.395404,0.942798,0.892972,0.000108,0.818783,0.080658,0.860197,Very High
681,696,0.507344,0.960822,0.884259,0.001905,0.812076,0.107751,0.868796,Very High
1040,1065,0.422980,0.969734,0.999229,0.000240,0.510060,0.402559,0.940889,Very High
1325,1360,0.563159,0.987588,0.989411,0.001100,0.724830,0.207391,0.927369,Very High


In [ ]:

time_data_features.sort_values(by="since_release_to_mean")

,movieId,mean_time,std_time,last_engagement,total_engagement,release_date_unix,since_release_to_mean,engagement_span,engagement_level
72026,232427,0.902808,0.003327,0.879230,0.000015,0.986586,0.000000,0.002271,Very Low
72494,234432,0.903314,0.000566,0.879230,0.000039,0.986586,0.000091,0.000523,Very Low
80075,268286,0.941885,0.000386,0.927345,0.000015,0.993293,0.000169,0.000286,Very Low
84666,282607,0.980466,0.000745,0.975696,0.000023,1.000000,0.000249,0.000568,Very Low
84722,282745,0.980570,0.000141,0.975684,0.000008,1.000000,0.000268,0.000076,Very Low
...,...,...,...,...,...,...,...,...,...
44306,166800,0.831976,0.117134,0.915825,0.000201,0.060399,0.938000,0.168817,Low
60998,202045,0.884441,0.077705,0.938393,0.000093,0.040260,0.968146,0.101661,Low
82047,275699,0.976620,0.048698,0.998821,0.000015,0.053693,0.970998,0.035992,Low
35849,148048,0.833139,0.161516,0.987890,0.000341,0.026846,0.972654,0.251697,High


In [ ]:
# rating aggrigator

#  Beräkning/Aggregering av ranking data för att ge ett standardizerat värde på ranking
#     * Lägga till alla typer av ranking värde 0.5 till 5.0 och antelet för varje ranking.
#     * En stadrizerad värde för medlet på av alla rankningar.
feature_list = ["0.5", "1.0", "1.5", "2.0", "2.5", "3.0", "3.5", "4.0", "4.5", "5.0"]
rating_count = rating_data.groupby("movieId")["rating"].value_counts()
rating_count = rating_count.rename("count")
rating_count = rating_count.to_frame()
rating_count.reset_index(level=["rating"], inplace=True)
rating_count["rating"] = rating_count["rating"].astype("str")

In [ ]:
rating_pivot = rating_count.pivot(columns="rating", values="count")
rating_pivot.reset_index(inplace=True)
rating_pivot = rating_pivot.fillna(0)
# rating_pivot["mean_rating"] = rating_pivot[feature_list].mean(axis=1)
rating_pivot.drop(feature_list, axis=1, inplace=True)
feature_list.append("mean_rating")

In [ ]:
rating_count["rating"] = rating_count["rating"].astype("str")

In [ ]:
rating_count.columns

Index(['rating', 'count'], dtype='object')

In [ ]:
# rating_pivot["mean_rating"] = scaler.fit_transform(rating_pivot["mean_rating"])

rating_pivot

rating,movieId
0,1
1,2
2,3
3,4
4,5
...,...
83234,288967
83235,288971
83236,288975
83237,288977


In [ ]:
for rating in feature_list:
    rating_pivot[rating] = rating_pivot[rating].astype(float)
weighted_sum = sum(float(r) * rating_pivot[r] for r in feature_list)
total_ratings = rating_pivot[feature_list].sum(axis=1)

# Compute weighted mean
rating_pivot["mean_rating_wieghted"] = weighted_sum / total_ratings
rating_pivot["total_ratings"] = total_ratings
# rating_pivot.drop(feature_list, inplace=True)

rating_pivot.sort_values(by="mean_rating_wieghted")

KeyError: '0.5'

In [ ]:
movies_data.head()

,movieId,title,year,genres,tf_idf_input,BoW_input
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,Adventure|Animation|Children|Comedy|Fantasy an...,"[[0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,..."
1,2,Jumanji,1995,Adventure|Children|Fantasy,Adventure|Children|Fantasy animals|based|on|a|...,"[[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,..."
2,3,Grumpier Old Men,1995,Comedy|Romance,Comedy|Romance sequel|moldy|old|old|age|old|me...,"[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,4,Waiting to Exhale,1995,Comedy|Drama|Romance,Comedy|Drama|Romance characters|chick|flick|gi...,"[[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."
4,5,Father of the Bride Part II,1995,Comedy,Comedy family|pregnancy|wedding|4th|wall|aging...,"[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [ ]:
tfidf = TfidfVectorizer(stop_words="english")

tfidf_matrix = tfidf.fit_transform(movies_data["tf_idf_input"])

print(tfidf_matrix.shape)

(86537, 51797)
